In [ ]:
import os
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from model import SpeechRecognition
from dataLoader import Data, collate_fn_padd
device = "cuda:0"

batchSize = 64
numWorkers = 8
def train_dataloader():
    d_params = Data.parameters
    train_dataset = Data(json_path="data/train.json", **d_params)
    return DataLoader(dataset=train_dataset,
                        batch_size=batchSize,
                        num_workers=numWorkers,
                        pin_memory=True,
                        collate_fn=collate_fn_padd)
def valid_dataloader():
    d_params = Data.parameters
    valid_dataset = Data(json_path="data/test.json", **d_params)
    return DataLoader(dataset=valid_dataset,
                        batch_size=batchSize,
                        num_workers=numWorkers,
                        pin_memory=True,
                        collate_fn=collate_fn_padd)

trainloader = train_dataloader()

In [ ]:
srmodel = SpeechRecognition()

import torch.optim as optim

optimizer = optim.AdamW(srmodel.parameters(), 1e-3)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min',
    factor=0.50, patience=6)
criterion = nn.CTCLoss(blank=28, zero_infinity=True)

In [ ]:
def forward(self, x, hidden):
        return self.model(x, hidden)

def step(model, batch):
    spectrograms, labels, input_lengths, label_lengths = batch 
    bs = spectrograms.shape[0]
    hidden = model._init_hidden(bs)
    hn, c0 = hidden[0].to(device), hidden[1].to(device)
    output, _ = model(spectrograms, hidden)
    output = F.log_softmax(output, dim=2)
    loss = criterion(output, labels, input_lengths, label_lengths)
    return loss

def training_step(batch):
    loss = step(batch)
    return {'loss': loss}

In [ ]:
losses = []
for epoch in range(100):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        loss = step(srmodel, data)
        losses.append(loss)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

import matplotlib.pyplot as plt
plt.plot(list(range(len(losses))),losses)